<a href="https://colab.research.google.com/github/junstones/ML-/blob/main/LDA_%EC%BD%94%EB%93%9C_%EC%98%B7_%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install KoNLPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 21.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
from collections import Counter # 단어 등장 횟수 카운트
from konlpy.tag import Okt # 형태소 분석기

In [ ]:
# 데이터 프레임 불러오기
df = pd.read_csv('comment_review_hanspell.csv')

# 불용어 리스트 불러오기
stopword_list = pd.read_excel('stopword_list.xlsx')
# 치환할 리스트 불러오기
replace_list = pd.read_excel('replace_list.xlsx')
# 한 글자인 키워드 리스트 불러오기
one_char_keyword = pd.read_excel('one_char_list.xlsx')

In [ ]:
# 단어 치환 함수
def replace_word(review):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in review:
                review = review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return review

# 불용어 제거 함수
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

In [ ]:
# 단어 치환
df['review_prep'] = ''
review_replaced_list = []
for review in tqdm(df['comment_review_hanspell']):
    review_replaced = replace_word(str(review)) # 문자열 데이터 변환
    review_replaced_list.append(review_replaced)
df['review_prep'] = review_replaced_list

# 한글 외 텍스트 제거
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), df['review_prep']))
df['review_prep'] = review_removed

# morphs 기반 토큰화
df_tokenized = list(map(lambda review: Okt().morphs(review,norm=True, stem=True), df['review_prep']))

# 불용어
df_removed_stopword = list(map(lambda tokens : remove_stopword(tokens), df_tokenized))

100%|██████████| 20226/20226 [00:51<00:00, 395.12it/s]


In [ ]:
len(df_removed_stopword)

20226

In [ ]:
# Gensim의 Dictionary 객체 생성
dictionary = corpora.Dictionary(df_removed_stopword)
list(dictionary.token2id.items())[:10]

[('가격', 0),
 ('배송', 1),
 ('빨르다', 2),
 ('삼박자', 3),
 ('색감', 4),
 ('어우러지다', 5),
 ('자다', 6),
 ('좋다', 7),
 ('하모니', 8),
 ('합리', 9)]

In [ ]:
# 문서-단어 행렬을 생성
corpus = [dictionary.doc2bow(text) for text in df_removed_stopword]

In [ ]:
# 첫 번째 문장의 DTM
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [ ]:
# LDA 적용
from gensim.models import ldamodel

topicK = 7 # 토픽 7개로 설정
num_trains = 10 # 훈련 횟수

lda_model = ldamodel.LdaModel(corpus,
                              num_topics=topicK, # 선정한 토픽 수
                              id2word=dictionary,
                              passes=num_trains, # 학습 횟수
                              random_state=42) # seed값 고정

In [ ]:
# 토픽 별 단어 분포 확인
for k in range(topicK):
    print(lda_model.show_topic(k, topn=10)) # 토픽 하나당 10개씩 확인

[('마음', 0.11897718), ('들다', 0.088202946), ('디자인', 0.030442093), ('색상', 0.029827816), ('예쁘다', 0.028243955), ('없다', 0.025406512), ('듭니', 0.024206124), ('세탁', 0.023997815), ('좋다', 0.022528306), ('핏', 0.022143066)]
[('좋다', 0.112825036), ('구매', 0.07017041), ('가격', 0.06952434), ('이쁘다', 0.031037536), ('물건', 0.02822376), ('만족하다', 0.020074371), ('추천', 0.02006997), ('싶다', 0.018847197), ('자다', 0.018788395), ('퀄리티', 0.017619088)]
[('좋다', 0.06598533), ('입다', 0.049739316), ('사다', 0.04470248), ('구매', 0.042154483), ('기본', 0.033880115), ('자다', 0.027564876), ('역시', 0.025809806), ('살다', 0.020417338), ('싸다', 0.019479597), ('가성', 0.019470593)]
[('사이즈', 0.09821913), ('입다', 0.08072671), ('좋다', 0.06274252), ('자다', 0.05496782), ('맞다', 0.042659983), ('이쁘다', 0.041665066), ('편하다', 0.032524485), ('예쁘다', 0.029722081), ('핏', 0.029016174), ('크다', 0.024065617)]
[('배송', 0.13316317), ('빠르다', 0.06704612), ('좋다', 0.06448104), ('옷', 0.03678585), ('자다', 0.036589846), ('오다', 0.035610136), ('이쁘다', 0.034095544), ('디자인', 0.0219

In [ ]:
len(corpus)

20226

In [ ]:
# 매 리뷰 마다 어떤 토픽에 속해있는지에 대한 데이터가 담긴 리스트 만들기
target_topics = []
for i in range(len(corpus)):
  list = []
  for topic_idx, prob in lda_model[corpus[i]]:
    list.append(prob)
    max_number = max(list)
  target_topic = list.index(max_number)
  target_topics.append(target_topic)

In [ ]:
len(target_topics)

20226

In [ ]:
# 분포 확인
series = pd.Series(target_topics)
series.value_counts()

3    5155
1    3517
5    3321
2    2467
0    2104
4    1910
6    1752
dtype: int64

In [ ]:
df['Topic'] = series
df.head(10)

,comment_review,comment_review_hanspell,review_prep,Topic
0,색감도 좋고 가격도 합리적이고 배송도 빨랐네요.\n 삼박자의 하모니가 잘 어우러졌습니다,색감도 좋고 가격도 합리적이고 배송도 빨랐네요. 삼박자의 하모니가 잘 어우러졌습니다,색감도 좋고 가격도 합리적이고 배송도 빨랐네요 삼박자의 하모니가 잘 어우러졌습니다,1
1,세일할때 사서 싸게 구매했어요 귀엽고 잘 입을 것 같아영,세일할 때 사서 싸게 구매했어요 귀엽고 잘 입을 것 같아요,세일할 때 사서 싸게 구매했어요 귀엽고 잘 입을 것 같아요,2
2,타임세일로 저렴하게 구매했어요!\n 우선 엄청얇은 니트입니다 탄탄한니트절태아니고 흐...,타임 세일로 저렴하게 구매했어요! 우선 엄청 얇은 니트입니다 탄탄한 니트 절대 아니...,타임 세일로 저렴하게 구매했어요 우선 엄청 얇은 니트입니다 탄탄한 니트 절대 아니고...,3
3,촉감도 부들부들하고 배송도 빠르고 실제로 보니 색상도 예뻐용,촉감도 부들부들하고 배송도 빠르고 실제로 보니 색상도 예뻐요,촉감도 부들부들하고 배송도 빠르고 실제로 보니 색상도 예뻐요,4
4,s 사려다가 그 사이에 품절돼서 m 샀는데 완전 마음에 들어요 안에 옷 껴입기에 품...,s 사려다가 그 사이에 품절돼서 m 샀는데 완전 마음에 들어요 안에 옷 껴입기에 품...,사려다가 그 사이에 품절돼서 샀는데 완전 마음에 들어요 안에 옷 껴입기에 품 적...,0
5,저렴한 가격으로 잘 산 것 같습니다 배송 빨라서 좋아요,저렴한 가격으로 잘 산 것 같습니다 배송 빨라서 좋아요,저렴한 가격으로 잘 산 것 같습니다 배송 빨라서 좋아요,4
6,무난하게 잘 입고 다닐 수 있을 것 같아여. 지퍼가 약간 빡빡 하지만 저렴하게 구매...,무난하게 잘 입고 다닐 수 있을 것 같아요. 지퍼가 약간 빡빡하지만 저렴하게 구매했...,무난하게 잘 입고 다닐 수 있을 것 같아요 지퍼가 약간 빡빡하지만 저렴하게 구매했으...,5
7,주머니 없는게 아쉽긴한데 색감도 좋고 너무 마음에\n 들어요 저렴한 가격에 잘 구매...,주머니 없는 게 아쉽긴 한데 색감도 좋고 너무 마음에 들어요 저렴한 가격에 잘 구매...,주머니 없는 게 아쉽긴 한데 색감도 좋고 너무 마음에 들어요 저렴한 가격에 잘 구매...,1
8,옷 두께는 살짝 얇고 재질은 부드럽지는 않아요!\n 처음엔 살짝 공장 냄새가 나는 ...,옷 두께는 살짝 얇고 재질은 부드럽지는 않아요! 처음엔 살짝 공장 냄새가 나는 것 ...,옷 두께는 살짝 얇고 재질은 부드럽지는 않아요 처음엔 살짝 공장 냄새가 나는 것 같...,5
9,옷 무겁지않아서 좋고 초여름 초가을에도 입을수있을듯\n 색감 밝은편이라 좋네요\n ...,옷 무겁지 않아서 좋고 초여름 초가을에도 입을 수 있을 듯 색감 밝은 편이라 좋네요...,옷 무겁지 않아서 좋고 초여름 초가을에도 입을 수 있을 듯 색감 밝은 편이라 좋네요...,1


In [ ]:
df.to_csv('comment_review_topic.csv', index=False, encoding='utf-8-sig')